# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import torch
import torch.utils.data
import torch.nn as nn
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [137]:
!ls -la "/srv/app/model/data/"

total 100
drwxr-xr-x. 2 root root    37 May  7 21:30 .
drwxr-xr-x. 4 root root  4096 May  7 19:30 ..
-rw-r--r--. 1 root root  6148 Jul 26  2019 .DS_Store
-rw-r--r--. 1 root root 87096 May  8 00:52 dga.pt


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.15.4
pandas version: 0.25.1


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'data'

In [4]:
# take a look at some matsnu example domains
from dga import matsnu

for i in range(10):
    print(matsnu.generate_domain())

leathercountermouthcommunicate.com
sidecombinejokedisappointed.com
advantageadmireborderinvite.com
hatabusestrugglereservebook.com
futurepostaddressplaypage.com
numberadaptpotcommentprint.com
creamlookdetermineappropriate.com
newsweredetailpopestimate.com
winnercoatbitwakespitewarn.com
ballclockreasontripbuild.com


In [5]:
# matsnu domains
matsnu_list = []

for i in range(20000):
    matsnu_list.append(matsnu.generate_domain())
    
matsnu_df = pd.DataFrame(matsnu_list, columns=['domain'])

print("Matsnu Shape:", matsnu_df.shape)

matsnu_df['label'] = 'dga'

matsnu_df.head()

Matsnu Shape: (20000, 1)


,domain,label
0,championshiptextorganize.com,dga
1,considerationbecomesalary.com,dga
2,screenawarddecidecampaign.com,dga
3,newsdigexerciseoilsolvereceive.com,dga
4,spotamazinginvitedevilconcern.com,dga


In [6]:
# alex top 1 million domains
alexa_df = pd.read_csv(data_dir + "/alexa_top_1m.csv", names=['domain'])

alexa_df['label'] = 'benign'

alexa_df.head()

,domain,label
1,google.com,benign
2,youtube.com,benign
3,facebook.com,benign
4,baidu.com,benign
5,wikipedia.org,benign


In [7]:
train_df = pd.concat([alexa_df.iloc[:20000], matsnu_df.iloc[:20000]], axis=0).reset_index(drop=True)

In [8]:
train_df.head()

,domain,label
0,google.com,benign
1,youtube.com,benign
2,facebook.com,benign
3,baidu.com,benign
4,wikipedia.org,benign


In [9]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
le = LabelEncoder()

In [10]:
# Apply le on categorical feature columns
train_df['label'] = le.fit_transform(train_df['label'])

train_df['label'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [145]:
# Training dataframe to csv
alexa_df = pd.read_csv(
    os.path.join(data_dir, 'alexa_top_1m.csv'),
    names=['domain']
).iloc[:10000]
alexa_df['label'] = 'benign'

matsnu_df = pd.DataFrame(
    [matsnu.generate_domain() for i in range(10000)],
    columns = ['domain']
)
matsnu_df['label'] = 'dga'

data_df = pd.concat([alexa_df, matsnu_df]).reset_index(drop=True)

class_labeler = LabelEncoder()

data_df['class'] = class_labeler.fit_transform(data_df['label'])

data_df.to_csv(os.path.join(data_dir, "dga.csv"), index=False)

In [146]:
data_df

,domain,label,class
0,google.com,benign,0
1,youtube.com,benign,0
2,facebook.com,benign,0
3,baidu.com,benign,0
4,wikipedia.org,benign,0
...,...,...,...
19995,sentencezoneinvestigatemeet.com,dga,1
19996,pressureriskopensmartwise.com,dga,1
19997,stresstrackbitesunthinkalarm.com,dga,1
19998,painfarmgradefalladaptempty.com,dga,1


In [148]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from __future__ import unicode_literals, print_function, division
from io import open
import glob

# print(findFiles('data/names/*.txt'))

import unicodedata
import string

class DomainDataset(Dataset):
    def __init__(self, data_dir, csv_filename, train=True):
        """
        Args:
            data_dir (string): directory name
            csv_filename (string): csv filename
        """
        
        self.data_df = pd.read_csv(os.path.join(data_dir, csv_filename))
        
        self.all_chars =  self.__build__chars__()
        self.inputs = self.data_df.iloc[:, 0]
                                   
        self.train = train
                                   
        if self.train:
            self.labels = self.data_df.iloc[:, -1]
        
        self.data_len = len(self.data_df.index)

    def __build__chars__(self):
        """Build dictionary of chars."""
        all_letters = string.ascii_letters + string.digits + " .'-"
        return {all_letters[i]:i+1 for i in range(0, len(all_letters))}
    
    def char_to_ix(self, char):
        """Character to index lookup."""
        return self.all_chars[char]

    def ix_to_char(self, char):
        """Index to character lookup."""
        for i, val in self.all_chars.items():
            if val == char:
                return i

    def domain_to_ix(self, domain):
        """Domain to sequence of indexes."""
        return torch.LongTensor([self.char_to_ix(i) for i in domain])

    def __getitem__(self, index):
        domain = self.domain_to_ix(self.inputs[index])
        if self.train:
            target = torch.Tensor([self.labels[index]])
            return domain, target
        return domain

    def __len__(self):
        return self.data_len

In [151]:
len(DomainDataset('data', 'alexa_top_1m.csv').all_chars)

66

In [96]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

    return xx_pad, yy, x_lens, y_lens

def _get_train_test_data_loader(batch_size, data_dir, filename, n_records):
    print("Getting test and train data loaders.")
    
    dataset =  DomainDataset(data_dir, filename)
    
    train_size = int(0.8 * len(dataset))
    
    test_size = len(dataset) - train_size
    
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
    
    test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)

    return train_dl, test_dl

def pad_collate_pred(batch):

    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

    return xx_pad, yy, x_lens, y_lens

def _get_predict_loader(batch_size, data_dir, filename):
    print("Getting test and train data loaders.")
    
    dataset =  DomainDataset(data_dir, filename)
    
    predict_dl = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
    
    return train_dl, test_dl

In [97]:
import unittest

class TestDGADataSet(unittest.TestCase):
    def setUp(self):
        self.n = 200
        self.test_dga = DGADataset('data', 'alexa_top_1m.csv', self.n)

    def test_length(self):
        actual_len = self.test_dga.__len__()
        self.assertEqual(actual_len, self.n * 2, 
                         "DataSet is not the right length ({}), should be {}".format(actual_len, 400)
                        )

    def test_labels(self):
        expected_labels = ['benign', 'dga']
        labels = self.test_dga.data_df['label'].unique().tolist()
        self.assertEqual(labels, 
                         expected_labels, 
                         "Incorrect labels ({}), expected {}".format(labels, expected_labels)
                        )
        
    def test_classes(self):
        expected_classes = [0, 1]
        classes = self.test_dga.data_df['class'].unique().tolist()
        self.assertEqual(classes, 
                         expected_classes, 
                         "Incorrect classes ({}), expected {})".format(classes, expected_classes)
                        )
        
    def test_index(self):
        index = self.test_dga.data_df.index.tolist()
        self.assertEqual(len(index), 
                         len(set(index)),
                         "Index contains duplicates ({}), expected {})".format(len(set(index)), len(index)))

    def test_getitem(self):
        item = self.test_dga.__getitem__(0)
        self.assertEqual(len(item), 
                         2,
                         "Get item output is wrong length ({}), expected {})".format(len(item), 2))
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 4.339s

OK


In [98]:
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

class DGAClassifier(nn.Module):
    """
    RNN Estimator for generating sequences of target variables.
    """
    
    def __init__(self, input_features=65, hidden_dim=12, n_layers=2, output_dim=1, embedding_dim=5, batch_size=10):
        super(DGAClassifier, self).__init__()

        # Variables
        self.hidden_dim = hidden_dim
        self.hidden_layers = n_layers
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        
        # Embedding
        self.embedding = nn.Embedding(input_features, embedding_dim)
        
        # RNN Layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=0.3, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.sigmoid = nn.Sigmoid()
        
#     def init_hidden(self):
#         """
#         Initialize the hidden and cell states of the LSTM with zeros.
#         """
#         return torch.zeros(self.hidden_layers, self.batch_size, self.hidden_dim)
        
    def forward(self, x, x_lens):
        """
        Perform a forward pass of our model on batch of tracks.
        """
        # x: (batch_size, longest_sequence, embedding) i.e. 10, 32, 5
        # hidden size: (hidden_layers, batch_size, hidden_dim) i.e. 2, 10, 30
        batch_size, seq_len = x.size()

        # x_embed: (batch_size, longest_sequence, 1?, embedding_size)
        embed_x = self.embedding(x)
        
        x_packed = pack_padded_sequence(embed_x, x_lens, batch_first=True, enforce_sorted=False)

        # Passing in the input and hidden state into the model and obtaining outputs
        output_packed, hidden_state = self.rnn(x_packed)
        
        output_padded, lengths = pad_packed_sequence(output_packed, batch_first=False)
        
        output = output_padded.view(batch_size*seq_len, self.hidden_dim)
        
        adjusted_lengths = [(l-1)*batch_size + i for i, l in enumerate(lengths)]
        
        lengthTensor = torch.tensor(adjusted_lengths, dtype=torch.int64)
        
        output = output.index_select(0, lengthTensor)
        
        output = output.view(batch_size, self.hidden_dim)
        
        output = self.sigmoid(self.fc(output))
        
        return output

In [99]:
def seed_everything(seed, cuda=False):
    # Set the random seed manually for reproducibility.
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

In [100]:
# TO:DO Figure out splunk query
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
# def stage(name):
#     with open("data/"+name+".csv", 'r') as f:
#         df = pd.read_csv(f)
#     with open("data/"+name+".json", 'r') as f:
#         param = json.load(f)
#     return df, param

batch_size = 32
num_epochs = 2
data_dir = 'data'
filename = 'alexa_top_1m.csv'
n_records = 5000

In [101]:
# # THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
train, test = _get_train_test_data_loader(batch_size, data_dir, filename, n_records)# print(df.describe())
# print(df)
param = {'feature_variables': ['domain'], 'target_variables': ['benign', 'dga']}
print(param)

Getting test and train data loaders.
{'feature_variables': ['domain'], 'target_variables': ['benign', 'dga']}


In [102]:
train.dataset

## Stage 2 - create and initialize a model

In [105]:
def init(df, param):
#     mapping = { key: value for value,key in enumerate(np.unique(Y.to_numpy().reshape(-1))) }

    
    input_size = 67
    hidden_dim = 30
    num_classes = 1
    learning_rate = 0.001
    embedding_dim = 5
    n_layers = 2

    model = {
        "input_size": input_size,
        "hidden_dim": hidden_dim,
        "embedding_dim": embedding_dim,
        "num_classes": num_classes,
        "n_layers": n_layers,
        "learning_rate": learning_rate,
#         "mapping": mapping,
        "num_epochs": num_epochs,
        "batch_size": batch_size,
    }
    
    model["train_loader"], model["test_loader"] = _get_train_test_data_loader(batch_size, data_dir, filename, n_records)

    print("FIT build RNN model with input size " + str(model["train_loader"].dataset.__len__()))
    print("FIT build model with target classes " + str(num_classes))
    
#     if 'options' in param:
#         if 'params' in param['options']:
#             if 'epochs' in param['options']['params']:
#                 model['num_epochs'] = int(param['options']['params']['epochs'])
#             if 'batch_size' in param['options']['params']:
#                 model['batch_size'] = int(param['options']['params']['batch_size'])

    # RNN Layer
    model['model'] = DGAClassifier(
        input_features=model["input_size"], 
        hidden_dim=model["hidden_dim"], 
        n_layers=model["n_layers"], 
        output_dim=model["num_classes"],
        embedding_dim=model["embedding_dim"], 
        batch_size=model["batch_size"]
    )
    
    # Define loss and optimizer
    model['criterion'] = torch.nn.BCELoss()
    
    model['optimizer'] = torch.optim.Adam(model['model'].parameters())
    
    return model

In [106]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

Getting test and train data loaders.
FIT build RNN model with input size 8000
FIT build model with target classes 1
{'input_size': 67, 'hidden_dim': 30, 'embedding_dim': 5, 'num_classes': 1, 'n_layers': 2, 'learning_rate': 0.001, 'num_epochs': 2, 'batch_size': 32, 'train_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fca01d82f98>, 'test_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fca01d82da0>, 'model': DGAClassifier(
  (embedding): Embedding(67, 5)
  (rnn): RNN(5, 30, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (sigmoid): Sigmoid()
), 'criterion': BCELoss(), 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)}


In [22]:
# def make_target(label, label_to_ix):
#     return torch.LongTensor([label_to_ix[label]])

# label_to_ix = {"benign": 0, "dga": 1}

In [23]:
# all_categories = ["benign", "dga"]
# def categoryFromOutput(output):
#     top_n, top_i = output.topk(1)
#     category_i = top_i[0].item()
#     return all_categories[category_i], category_i

In [24]:
# def binary_acc(y_pred, y_test):
#     y_pred_tag = torch.round(y_pred)

#     correct_results_sum = (y_pred_tag == y_test).sum().float()
#     acc = correct_results_sum/y_test.shape[0]
#     acc = torch.round(acc * 100)
    
#     return acc

In [62]:
def update_stats(accuracy, confusion_matrix, output, y):
    output = torch.round(output).flatten()
    equal = torch.eq(output, y)
    correct = int(torch.sum(equal))
    for j, i in zip(output, y):
        confusion_matrix[int(i),int(j)]+=1

    return accuracy + correct, confusion_matrix

## Stage 3 - fit the model

In [132]:
import time

# train your model
# returns a fit info json object and may modify the model object
def fit(model, param):
    
    returns = {"message": "model trained"}

    cuda = torch.cuda.is_available()
    device = torch.device("cpu") if not cuda else torch.device("cuda:0")
    seed_everything(seed=1337, cuda=cuda)
    
    accuracy, confusion_matrix = 0, np.zeros((2, 2), dtype=int)
    
    for epoch in range(1, model['num_epochs'] + 1):

        # Train
        model['model'].train()
        print("Train ({})".format(epoch))
        print("-"*20)
        t = time.time()

        accuracy, confusion_matrix = 0, np.zeros((2, 2), dtype=int)
        
        total_loss = 0
        
        # Iterate over dataset
        for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["train_loader"]):
            
            # Clear stored gradient
            model["optimizer"].zero_grad()

            # Forward pass
            output =  model['model'](x_padded, x_lens)
            
            loss = model['criterion'](output, torch.Tensor(y_padded).squeeze(0))

            total_loss += float(loss)
            
            accuracy, confusion_matrix = update_stats(
                accuracy, 
                confusion_matrix, 
                torch.Tensor(output), 
                torch.Tensor(y_padded)
            )
            
            # Backward pass
            loss.backward()

            # Update parameters
            model["optimizer"].step()
                            
            print("[Batch]: {}/{} in {:.5f} seconds".format(batch_num, len(model["train_loader"]), time.time() - t), end='\r', flush=True)
            t = time.time()
            
        print("")
        print("[Loss]: {:.5f}".format(total_loss / len(model["train_loader"])))
        print("[Accuracy]: {}/{} : {:.3f}%".format(
            accuracy, len(model["train_loader"].dataset), accuracy / len(model["train_loader"].dataset) * 100))
        print(confusion_matrix, "\n")
        
        # Evaluate
        model['model'].eval()
        accuracy, confusion_matrix = 0, np.zeros((2, 2), dtype=int)
        t = time.time()
        total_loss = 0
        print("Validation ({})".format(epoch))
        print("-"*20)
        with torch.no_grad():
            for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["test_loader"]):
                output =  model['model'](x_padded, x_lens)
                total_loss += float(model['criterion'](output, torch.Tensor(y_padded).squeeze(0)))
                accuracy, confusion_matrix = update_stats(
                    accuracy, 
                    confusion_matrix, 
                    torch.Tensor(output), 
                    torch.Tensor(y_padded)
                )
                print("[Batch]: {}/{} in {:.5f} seconds".format(batch_num, len(model["test_loader"]), time.time() - t), end='\r', flush=True)
                t = time.time()
                
        print("")
        print("[Loss]: {:.5f}".format(total_loss / len(model["test_loader"])))
        print("[Accuracy]: {}/{} : {:.3f}%".format(
            accuracy, len(model["test_loader"].dataset), accuracy / len(model["test_loader"].dataset) * 100))
        print(confusion_matrix, "\n")
        
    # memorize parameters
    returns['model_epochs'] = model['num_epochs']
    returns['model_batch_size'] = model['batch_size']
    returns['model_loss_acc'] = total_loss / len(model["train_loader"])
            
    return returns

In [133]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,param))

Train (1)
--------------------
[Batch]: 249/250 in 0.03208 seconds
[Loss]: 0.00405
[Accuracy]: 7994/8000 : 99.925%
[[3987    5]
 [   1 4007]] 

Validation (1)
--------------------
[Batch]: 62/63 in 0.00597 seconds
[Loss]: 0.00404
[Accuracy]: 1999/2000 : 99.950%
[[1007    1]
 [   0  992]] 

Train (2)
--------------------
[Batch]: 249/250 in 0.06303 seconds
[Loss]: 0.01950
[Accuracy]: 7965/8000 : 99.562%
[[3970   22]
 [  13 3995]] 

Validation (2)
--------------------
[Batch]: 62/63 in 0.01706 seconds
[Loss]: 0.01838
[Accuracy]: 1993/2000 : 99.650%
[[1001    7]
 [   0  992]] 

{'message': 'model trained', 'model_epochs': 2, 'model_batch_size': 32, 'model_loss_acc': 0.0046313053680351}


In [ ]:
classes = ('benign', 'dga')

In [ ]:
# # # print images
# # imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
net = RNNEstimator()
net.load_state_dict(rnn, strict=False)
net.eval()

dataiter = iter(testLoader)
domains, labels, xx_lens, _ = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(len(labels))))

def evaluate(domains):
    with torch.no_grad():
        for data in testloader:
            input = domains
            hidden = model['model'].init_hidden()
            output = model['model'](input, xx_lens, hidden)[0]
            y_hat = output[:, -1, :]
            print(y_hat)
            _, predicted = torch.max(y_hat, 1)
            prediction = [classes[key] for key in predicted.numpy()]
        
evaluate(domains)

## Stage 4 - apply the model

In [ ]:
#TO-DO: How does Splunk stage data for apply method. In a CSV?
def apply(model, df, param):
    
    test_df = _get_train_data_loader(batch_size, data_dir, filename, 100)# print(df.describe())
    
    classes = {v: k for k, v in model['mapping'].items()}
    
    # Evaluate
    model['model'].eval()
    print("Validation ({})".format(epoch))
    print("-"*20)
    with torch.no_grad():
        for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["test_loader"]):
            output =  model['model'](x_padded, x_lens)
            output = torch.round(output).flatten()
    return batch_predictions

In [190]:
# net = RNNEstimator()
# net.load_state_dict(torch.load(PATH))

# load(name)

In [191]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

Get train data loader.
torch.Size([10, 35, 2])
tensor([[0.2936, 0.7276],
        [0.0718, 0.9395],
        [0.0216, 0.9853],
        [0.0138, 0.9911],
        [0.0142, 0.9907],
        [0.0156, 0.9898],
        [0.0139, 0.9908],
        [0.0146, 0.9904],
        [0.0175, 0.9885],
        [0.0152, 0.9901],
        [0.0131, 0.9914],
        [0.0144, 0.9902],
        [0.0159, 0.9896],
        [0.0136, 0.9912],
        [0.0135, 0.9913],
        [0.0136, 0.9912],
        [0.0159, 0.9897],
        [0.0143, 0.9907],
        [0.0161, 0.9894],
        [0.0144, 0.9905],
        [0.0146, 0.9906],
        [0.0135, 0.9913],
        [0.0134, 0.9912],
        [0.0147, 0.9904],
        [0.0140, 0.9909],
        [0.0143, 0.9906],
        [0.0145, 0.9903],
        [0.0154, 0.9902],
        [0.0125, 0.9919],
        [0.5005, 0.5264],
        [0.5005, 0.5264],
        [0.5005, 0.5264],
        [0.5005, 0.5264],
        [0.5005, 0.5264],
        [0.5005, 0.5264]])
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


## Stage 5 - save the model

In [178]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    torch.save(model, MODEL_DIRECTORY + name + ".pt")
    return model

In [179]:
save(model, 'dga')

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNNEstimator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


{'input_size': 66,
 'hidden_dim': 30,
 'embedding_dim': 5,
 'num_classes': 2,
 'n_layers': 2,
 'learning_rate': 0.001,
 'mapping': {0: 0, 1: 1},
 'num_epochs': 5,
 'batch_size': 10,
 'train_loader': <torch.utils.data.dataloader.DataLoader at 0x7fd5cc17cf98>,
 'model': RNNEstimator(
   (embedding): Embedding(66, 5)
   (rnn): RNN(5, 30, num_layers=2, batch_first=True, dropout=0.3)
   (fc): Linear(in_features=30, out_features=2, bias=True)
 ),
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 )}

In [182]:
rnn = load('dga')

## Stage 6 - load the model

In [181]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = torch.load(MODEL_DIRECTORY + name + ".pt")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return model summary
def summary(model=None):
    returns = {"version": {"pytorch": torch.__version__} }
    if model is not None:
        if 'model' in model:
            returns["summary"] = str(model)
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code